In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import catboost as cb
from catboost import Pool, CatBoostRegressor, CatBoostClassifier
import matplotlib.pyplot as plt
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer,  StandardScaler, LabelEncoder

import sklearn.decomposition, sklearn.linear_model, sklearn.pipeline, sklearn.metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

import pickle
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error
from sklearn.metrics import classification_report

pd.set_option('display.max_rows', 100)
#np.set.printoptions(prediction = 4)

%matplotlib inline

In [2]:
#df = pd.read_csv('data/pokemon-go.csv')


df = pd.read_csv('data/pokemon_go.csv')
df_pokedex_name = pd.read_csv('data/pokedex.csv')

In [ ]:
df_pokedex_name.shape

In [ ]:
df_pokedex_name.head()

In [ ]:
df_pokedex_name.isnull().sum()

In [3]:
df1 = df_pokedex_name.drop('img', axis = 1)
df2 = df1.drop('wiki', axis = 1)
df3 = df2.drop('type_2', axis = 1)


In [ ]:
df3.info()

In [ ]:
df['local_time'].value_counts()

In [ ]:
df.shape

In [4]:
target = 'pokedex_id'
y = df[target]


In [ ]:
len(y)

In [5]:
X = df.drop(target, axis = 1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

In [7]:
split = X_train['local_time'].str.split('T', 1, expand= True)
split_1 = X_test['local_time'].str.split('T', 1, expand= True)

In [9]:
#split_1

In [10]:
X_split_train  = X_train.assign(first_part=split[0], last_part=split[1])
X_split_test  = X_test.assign(first_part=split_1[0], last_part=split_1[1])

In [12]:
#X_split_test

In [13]:
X_split_train.drop('local_time', 1, inplace=True)
X_split_test.drop('local_time', 1, inplace=True)

In [14]:
X_train = X_split_train
X_test = X_split_test

In [16]:
#X_test

In [17]:
X_train = X_train.rename(columns = {'last_part':'local_time'})
X_test = X_test.rename(columns = {'last_part':'local_time'})

In [18]:
X_train = X_train.rename(columns = {'first_part':'week_day'})
X_test = X_test.rename(columns = {'first_part':'week_day'})

In [20]:
X_test

,latitude,longitude,close_to_water,city,weather,temperature,population_density,week_day,local_time
115216,45.521321,9.596123,False,Rome,MostlyCloudy,22.7,591.089900,2016-09-05,21:32:40
87289,47.647997,-122.301393,True,Los_Angeles,Clear,14.8,4842.162600,2016-09-06,07:01:43
43507,41.672363,2.785256,True,Madrid,Clear,22.8,0.000000,2016-09-06,21:25:37
247716,49.973351,16.395799,False,Prague,MostlyCloudy,22.9,64.717780,2016-09-03,16:45:31
184245,44.079942,-103.228522,False,Denver,PartlyCloudy,18.8,790.019500,2016-09-04,20:20:57
...,...,...,...,...,...,...,...,...,...
275812,4.641414,114.438080,True,Brunei,MostlyCloudy,31.5,18.862122,2016-09-03,06:26:28
40230,39.744421,-84.083999,False,New_York,Clear,29.7,570.040200,2016-09-06,22:13:39
121026,50.714452,-1.986068,True,London,Clear,19.5,1886.212200,2016-09-05,19:38:14
73274,38.986460,-84.695671,False,New_York,Clear,19.9,266.193150,2016-09-06,11:37:11


In [21]:
 X_train['day_of_week']  = pd.to_datetime(X_train['week_day']).dt.weekday_name

In [ ]:
#X_train['day_of_week'] = (X_train['week_day']).dt.weekday_name

In [22]:
 X_test['day_of_week']  = pd.to_datetime(X_test['week_day']).dt.weekday_name

In [23]:
X_train.drop('latitude', 1, inplace=True)
X_test.drop('latitude', 1, inplace=True)

In [24]:
X_train.drop('longitude', 1, inplace=True)
X_test.drop('longitude', 1, inplace=True)

In [25]:
X_test

,close_to_water,city,weather,temperature,population_density,week_day,local_time,day_of_week
115216,False,Rome,MostlyCloudy,22.7,591.089900,2016-09-05,21:32:40,Monday
87289,True,Los_Angeles,Clear,14.8,4842.162600,2016-09-06,07:01:43,Tuesday
43507,True,Madrid,Clear,22.8,0.000000,2016-09-06,21:25:37,Tuesday
247716,False,Prague,MostlyCloudy,22.9,64.717780,2016-09-03,16:45:31,Saturday
184245,False,Denver,PartlyCloudy,18.8,790.019500,2016-09-04,20:20:57,Sunday
...,...,...,...,...,...,...,...,...
275812,True,Brunei,MostlyCloudy,31.5,18.862122,2016-09-03,06:26:28,Saturday
40230,False,New_York,Clear,29.7,570.040200,2016-09-06,22:13:39,Tuesday
121026,True,London,Clear,19.5,1886.212200,2016-09-05,19:38:14,Monday
73274,False,New_York,Clear,19.9,266.193150,2016-09-06,11:37:11,Tuesday


In [ ]:
### save modified X and y  data frames
#export_csv = df.to_csv (r'data\modified_pokemon_go.csv', index = None, header=True) 

In [ ]:
X_test['local_time']

In [26]:
split_time_train = X_train['local_time'].str.split(':', 1, expand= True)
split_time_test = X_test['local_time'].str.split(':', 1, expand= True)

X_train_split2 = X_train.assign(first_part=split_time_train[0], last_part=split_time_train[1])
X_test_split2 = X_test.assign(first_part=split_time_test[0], last_part=split_time_test[1])

X_train_split2.drop('local_time', 1, inplace=True)
X_test_split2.drop('local_time', 1, inplace=True)

X_train_split2.drop('last_part', 1, inplace=True)
X_test_split2.drop('last_part', 1, inplace=True)

X_train = X_train_split2
X_test = X_test_split2

X_train = X_train.rename(columns = {'first_part':'local_time'})
X_test = X_test.rename(columns = {'first_part':'local_time'})

In [27]:
X_train

,close_to_water,city,weather,temperature,population_density,week_day,day_of_week,local_time
10058,True,Stockholm,MostlyCloudy,18.2,43.430546,2016-09-07,Wednesday,21
237759,True,Stockholm,MostlyCloudy,14.7,43.430546,2016-09-03,Saturday,21
185989,False,Los_Angeles,Clear,25.3,591.089900,2016-09-04,Sunday,19
145608,False,Rome,MostlyCloudy,23.4,1411.257600,2016-09-05,Monday,10
121648,True,Paris,PartlyCloudy,24.5,0.000000,2016-09-05,Monday,19
...,...,...,...,...,...,...,...,...
172101,False,New_York,Clear,15.9,659.018550,2016-09-05,Monday,01
40544,False,Indianapolis,Clear,30.3,286.215330,2016-09-06,Tuesday,22
96439,False,Mexico_City,Clear,25.6,2431.234100,2016-09-06,Tuesday,03
7516,False,Monterrey,Clear,32.7,5804.697000,2016-09-07,Wednesday,22


In [28]:
X_test

,close_to_water,city,weather,temperature,population_density,week_day,day_of_week,local_time
115216,False,Rome,MostlyCloudy,22.7,591.089900,2016-09-05,Monday,21
87289,True,Los_Angeles,Clear,14.8,4842.162600,2016-09-06,Tuesday,07
43507,True,Madrid,Clear,22.8,0.000000,2016-09-06,Tuesday,21
247716,False,Prague,MostlyCloudy,22.9,64.717780,2016-09-03,Saturday,16
184245,False,Denver,PartlyCloudy,18.8,790.019500,2016-09-04,Sunday,20
...,...,...,...,...,...,...,...,...
275812,True,Brunei,MostlyCloudy,31.5,18.862122,2016-09-03,Saturday,06
40230,False,New_York,Clear,29.7,570.040200,2016-09-06,Tuesday,22
121026,True,London,Clear,19.5,1886.212200,2016-09-05,Monday,19
73274,False,New_York,Clear,19.9,266.193150,2016-09-06,Tuesday,11


In [29]:
X_train.drop('week_day', 1, inplace=True)
X_test.drop('week_day', 1, inplace=True)

In [31]:
X_test

,close_to_water,city,weather,temperature,population_density,day_of_week,local_time
115216,False,Rome,MostlyCloudy,22.7,591.089900,Monday,21
87289,True,Los_Angeles,Clear,14.8,4842.162600,Tuesday,07
43507,True,Madrid,Clear,22.8,0.000000,Tuesday,21
247716,False,Prague,MostlyCloudy,22.9,64.717780,Saturday,16
184245,False,Denver,PartlyCloudy,18.8,790.019500,Sunday,20
...,...,...,...,...,...,...,...
275812,True,Brunei,MostlyCloudy,31.5,18.862122,Saturday,06
40230,False,New_York,Clear,29.7,570.040200,Tuesday,22
121026,True,London,Clear,19.5,1886.212200,Monday,19
73274,False,New_York,Clear,19.9,266.193150,Tuesday,11


In [32]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207214 entries, 10058 to 18057
Data columns (total 7 columns):
close_to_water        207214 non-null bool
city                  207214 non-null object
weather               207214 non-null object
temperature           207214 non-null float64
population_density    207214 non-null float64
day_of_week           207214 non-null object
local_time            207214 non-null object
dtypes: bool(1), float64(2), object(4)
memory usage: 11.3+ MB


### Making a model

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(X_train['local_time'])
#le.fit(X_train['week_day'])
le.fit(X_test['local_time'])
#le.fit(X_test['week_day'])


In [ ]:
X_train.info()

In [ ]:
mapper = DataFrameMapper([
    (['city'], [LabelBinarizer()]),
    (['weather'], [LabelBinarizer()]),
    (['close_to_water'], [LabelBinarizer()]),
    (['temperature'], [StandardScaler()]),
    (['population_density'], [StandardScaler()]),
    (['week_day'], [LabelBinarizer()]),
    (['local_time'], [LabelBinarizer()]),
    ], df_out= True)

In [ ]:
mapper

In [ ]:
X_train.shape

In [ ]:
Z_train= mapper.fit(X_train)
#Z_test = mapper.transform(X_test)

In [ ]:
Z_train

In [ ]:
Z_train= mapper.transform(X_train)

In [ ]:
Z_test = mapper.transform(X_test)

### CAT BOOST CLASSIFIER

In [ ]:
model = cb.CatBoostClassifier(
    iterations=100, 
    early_stopping_rounds=10)
    custom_loss=['AUC', 'Accuracy'])

In [ ]:
model.fit(
    Z_train, 
    y_train,
    eval_set=(Z_test, y_test),
    verbose=False,
    plot=True)

In [ ]:
train_score = model.score(Z_train, y_train) # train (learn) score

In [ ]:
train_score

In [ ]:
val_score = model.score(Z_test, y_test) # val (test) score

In [ ]:
val_score